1. Installing dependencies and setup 


In [ ]:
from google.colab import drive 
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install tensorflow tensorflow-gpu opencv-python matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu
  error: subprocess-exited-with-error
  
  × Running setup.py install for tensorflow-gpu did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for tensorflow-gpu ... error
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> tensorflow-gpu

note: This is an issue with the package mentioned above, not pip.
hint: See above for outpu

In [ ]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.4
aiosignal                     1.3.1
alabaster                     0.7.13
albumentations                1.2.1
altair                        4.2.2
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.2.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.11.0
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        6.0.0
blis  

In [ ]:
import tensorflow as tf 
import os 

In [ ]:
gpus = tf.config.experimental.list_physical_devices('CPU')

In [ ]:
#limiting the memory growth by setting up GPU memory consumption growth which helps avoid OOM errors
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus : 
  tf.config.experimental.set_memory_growth(gpu, True)

2. Remove dodgy images 

In [ ]:
import cv2 
import imghdr
from matplotlib import pyplot as plt 

In [ ]:
data_dir = "/content/drive/My Drive/Colab Notebooks/Train Data/Normal/images_001/"

In [ ]:
os.listdir(data_dir)

['images', 'Normal ']

In [ ]:
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

In [ ]:
for image_class in os.listdir(data_dir):
  for image in os.listdir(os.path.join(data_dir, image_class)):
    image_path = os.path.join(data_dir, image_class, image)
    try: 
      img = cv2.imread(image_path)
      tip = imghdr.what(image_path)
      if tip not in image_exts:
          print("Image not in ext list {}".format(image_path))
          os.remove(image_path)
    except Exception as e:
      print("Issue with image{}".format(image_path)) 
    

3. Load Data 

In [ ]:
import numpy as np 
from matplotlib import pyplot as plt 

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('/content/drive/My Drive/Colab Notebooks/Train Data/Normal/images_001')

#Then calling image_dataset_from_directory(main_directory, labels='inferred') will return a tf.data.Dataset that 
#yields batches of images from the subdirectories class_a and class_b, together with labels 0 and 1 
#(0 corresponding to class_a and 1 corresponding to class_b).

Found 5653 files belonging to 3 classes.


In [ ]:
data_iterator = image.as_numpy_iterator()

AttributeError: ignored

In [ ]:
batch = data_iterator.next()

In [ ]:
#class 1 = Normal 
#Class 2 = Pnemonia 

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

3. Scale the data 

In [ ]:
data = data.map(lambda x,y: (x/255,y))

In [ ]:
scaled_iterator = data.as_numpy.iterator()

In [ ]:
batch = scaled_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4,figsize=(20,20))
for idx, img enumerate(batch[0],[:4]):
  ax[idx].imshow(img.astype(int)
  ax[idx].title.set_est(batch[1][idx])

2.2 SPLIT DATA 

In [ ]:
len(data)

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data*.1)

In [ ]:
train_size * val_size * test_size

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
train = data.skip(train_size*val_size).take(test_size)

In [ ]:
len(test)

3. DEEP LEARNING MODEL

In [ ]:
from tensorflow.keras.models import sequential
from tensorflow.keras.layers import COnv2d, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid')

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

Train

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)


In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

Plot performance 

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

Evaluate 

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

TEST

In [ ]:
import cv2

In [ ]:
img = cv2.imread('154006829.jpg')
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))


In [ ]:
yhat

In [ ]:
if yhat > 0.5: 
    print(f'Normal')
else:
    print(f'May have a disease like pnemonia, Please go to a doctor for further checkup ')

Save the model 

In [ ]:
from tensorflow.keras.models import load_model


In [ ]:
model.save(os.path.join('models','imageclassifier.h5'))


In [ ]:
new_model = load_model('imageclassifier.h5')


In [ ]:
new_model.predict(np.expand_dims(resize/255, 0))